In [1]:
# 모듈 불러오기
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
from jetbot import Robot

import cv2 as cv
import numpy as np
import uuid
import time
import os
import glob

In [2]:
# global 변수
car_move = 1

In [3]:
# Camera 객체 생성하기
camera = Camera.instance()

In [4]:
# Robot 객체 생성하기
robot = Robot()

In [5]:
# 위젯 등록하기
camera_image = widgets.Image(format='jpeg', width=200, height=200, layout=widgets.Layout(align_self='center'))
camera_image.value = bgr8_to_jpeg(np.zeros((200, 200, 3)))

In [14]:
# 버튼 프로시저
def procCamera(change):
    # camera_image 위젯이 이미지 출력하기
    camera_image.value = bgr8_to_jpeg(camera.value)
    
def procStart(change):
    global car_move
    
    # JetBot 정지 여부 설정하기
    car_move = 1
    
def procStop(change):
    global car_move
    
    # JetBot 정지 여부 설정하기
    car_move = 0

In [22]:
# 버튼 모양 정의하기

# 버튼 레이아웃 설정하고 버튼 생성하기
layoutButton = widgets.Layout(width='100px', height='80px', align_self='center')
button_camera = widgets.Button(description='Camera', layout=layoutButton)
button_start = widgets.Button(description='Start', layout=layoutButton)
button_stop = widgets.Button(description='Stop', layout=layoutButton)

# 버튼 이벤트 핸들러 설정하기
button_camera.on_click(procCamera)
button_start.on_click(procStart)
button_stop.on_click(procStop)

# 이미지 박스 만들고 이미지 출력하기
img_temp = np.ones((200, 200, 3), np.uint8)
box = widgets.VBox([camera_image, button_camera, button_start, button_stop], layout=widgets.Layout(align_self='center'))
display(box)

In [49]:
# JetBot 제어하기
def JetBotUp():
    robot.set_motors(0.1, 0.1)

def JetBotDown():
    robot.set_motors(-0.1, -0.1)

def JetBotLeft():
    robot.set_motors(0.1, -0.1)

def JetBotRight():
    robot.set_motors(-0.1, 0.1)
    
def JetBotStop():
    robot.set_motors(0.0, 0.0)
    robot.stop()
    

In [39]:
# 다각형 그리기
def DrawPoloygon(img_data, color, thickness, *pts):
    x = 0
    y = 0

    arr = list()

    # 좌표 리스트 만들기
    for i, pt in enumerate(pts):
        if i % 2 == 0:
            x = pt
        else:
            y = pt
            arr.append([x, y])
    
    # 점 배열 만들기
    arr_pts = np.array([arr])
    cv.polylines(img_data, [arr_pts], False, color, thickness)
    
# Gray 컬러를 Widget에서 보기
def ShowImageGray(img_data):
    # Gray -> BGR8
    img_bgr8 = cv.cvtColor(img_data, cv.COLOR_GRAY2BGR)
    
    # BGR8 -> jpeg
    img_jpeg = bgr8_to_jpeg(img_bgr8)
    
    camera_image.value = img_jpeg
    
def ShowImage(img_data):
    # BGR8 -> jpeg
    img_jpeg = bgr8_to_jpeg(img_data)
    
    camera_image.value = img_jpeg
    
def ShowText(img_data, x, y, strtext):
    font = cv.FONT_HERSHEY_COMPLEX
    scale = 0.5
    color = (255, 255, 255)
    thickness = 1
    cv.putText(img_data, strtext, (x, y), font, scale, color, thickness)

def GetMidPoint(img_data, debug_show = False):
    mid_point = 0.0
    
    if debug_show:
        img_blank = np.copy(img)
        img_blank[:] = 0

    # 외곽선 구하기
    contours, hierachy = cv.findContours(img_data, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    height_diff = 10
    max_area = 0
    for contour in contours:
        area = cv.contourArea(contour)
        
        if False:
            strtext = str(area)
            ShowText(img_blank, 0, height_diff, strtext)
            height_diff = height_diff + 40

        if area > 400 and area < 2000 and area > max_area:
            # 외곽선 출력하기
            if debug_show:
                color = (255, 0, 0)
                thickness = 1
                cv.drawContours(img_blank, contour, -1, color, thickness)

            # 외곽선 길이 구하기
            peri = cv.arcLength(contour, True)

            # 외곽선 구하기
            epsilon = 0.06 * peri
            approx = cv.approxPolyDP(contour, epsilon, True)

            # 경계 사각형 그리기
            x, y, w, h = cv.boundingRect(approx)

            # 좌표값 출력하고 사각형 그리기
            if debug_show:
                strtext = "{} {} {} {}".format(x, y, w, h)
                ShowText(img_blank, 0, height_diff, strtext)
                height_diff = height_diff + 40        
                cv.rectangle(img_blank, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # 너비와 높이 비율 검사하기
            if (h / w) > 2.0:
                mid_point = x + 0.5 * w
                
    if debug_show:
        ShowText(img_blank, 0, height_diff, str(mid_point))
        ShowImage(img_blank)

    return mid_point

In [18]:
# 이미지 변경하기
width = camera.value.shape[1]
height = camera.value.shape[0]

img = cv.resize(camera.value, (200, 200))

width = img.shape[1]
height = img.shape[0]

ShowImage(img)

In [47]:
def RunCar():    
    error_count = 0
    
    test_count = 0
    
    while car_move:
        # (1) 이미지 변경하기
        width = camera.value.shape[1]
        height = camera.value.shape[0]

        img = cv.resize(camera.value, (200, 200))

        width = img.shape[1]
        height = img.shape[0]

        # (2) 카메라 이미지 BINARY 변경하기
        img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        result, img_thresh = cv.threshold(img_gray, 90, 255, cv.THRESH_BINARY_INV)

        ShowImageGray(img_thresh)

        # (3) 크롭하기
        img_thresh[0:40, :] = 0
        img_thresh[160:, :] = 0

        ShowImageGray(img_thresh)

        # (4) 침식 연산 적용해 노이즈 제거하가
        kernel = np.ones((3, 3), np.uint8)
        img_erode = cv.erode(img_thresh, kernel, iterations=4)
        ShowImageGray(img_erode)
        
        # (6) 중심 좌표 구하기
        mid_point = GetMidPoint(img_thresh, True)
        
        # (7) JetBot 움직이기
#         if mid_point == 0.0:
#             error_count = error_count + 1
#             JetBotUp()
#             time.sleep(0.01)
        if mid_point < 90.0:
            error_count = 0
            
            JetBotRight()
            time.sleep(0.01)
            
            JetBotUp()
            time.sleep(0.01)
        elif mid_point > 110.0:
            error_count = 0
            
            JetBotLeft()
            time.sleep(0.01)
            
            JetBotUp()
            time.sleep(0.01)
        else:
            error_count = 0
            
            JetBotUp()
            time.sleep(0.01)
        
        if error_count > 5:
            JetBotUp()
            time.sleep(0.01)
            error_count = 0
            
        test_count = test_count + 1
        if test_count > 400:
            JetBotStop()
            break;


In [52]:
# 실행하기
RunCar()
robot.stop()

In [53]:
display(camera_image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…